# INSURANCE DASHBOARD

In [1]:
# import modules
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import seaborn as sns

%load_ext autoreload
%autoreload 2
import ipypivot as pt

%matplotlib inline

In [2]:
# load dataset
df = pd.read_csv('insurance.csv')

# explore dataset
#df.info()

In [3]:
# filter data by charges
# ALL will allow the user to remove the filter
ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    """
    Will allow the user to select uninque value
    when filtering the data or seel all the data
    """
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique


In [4]:
# create dropdowns to filter data frame by selected value
dropdown_bmi = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.bmi), description='BMI')

# capture widget out
output_bmi = widgets.Output()
plot_output = widgets.Output()


def dropdown_bmi_event(change):
    """Will allow user to access info
    based on whether the selection is ALL or
    unique value"""
    plot_output.clear_output()
    output_bmi.clear_output() #clears the previous output on each iteration
    with output_bmi:
        if (change.new == ALL):
            display(df)
        else:
            display(df[df.bmi == change.new])

    with plot_output:
        @interact
        def bar_plots():
            sns.set()
    
            plt.subplots(figsize=(15,10))
            plt.subplot(221)
            bar_sex = sns.barplot(x="sex", y="charges", data=df,
                 order=["male", "female"])
    
            plt.subplot(222)
            bar_region = sns.barplot(data=df, x="region", y="charges", 
   
                             order=["southeast", "southwest", "northeast", "northwest"])
            plt.subplot(223)
            bar_smoker = sns.barplot(data=df, x="smoker", y="charges", 
                     order=["yes", "no"])
    
            plt.subplot(224)
            scatter_bmi = sns.scatterplot(data=df, x="bmi", y="charges")
                     
# link the dropdown to the observe method which will take the invoked function filter through the data frame
dropdown_bmi.observe(dropdown_bmi_event, names='value')

display(dropdown_bmi)


Dropdown(description='BMI', options=('ALL', 15.96, 16.815, 17.195, 17.29, 17.385, 17.4, 17.48, 17.67, 17.765, …

In [5]:
#display(output_bmi)
tab = widgets.Tab([output_bmi, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'Data Viz')

display(tab)

In [6]:
def pivot_table():
    p = pt.PivotUI(df_data=df)

    opts = p.table.options

    dic = {
        'tpl': '$.pivotUtilities.aggregatorTemplates',
        'sortAs': '$.pivotUtilities.sortAs'
    }

    opts.rows = ['region']
    opts.cols = ['sex']

    opts.aggregators = {
        "Number of Clients": 'function() {{ return {tpl}.count()() }}'.format(**dic),
        "Average Age of Clients": 'function() {{ return {tpl}.average()(["age"]) }}'.format(**dic),
        "Total Client Fees": 'function() {{ return {tpl}.sum()(["charges"]) }}'.format(**dic),
        "Average Client Fees": 'function() {{ return {tpl}.average()(["charges"]) }}'.format(**dic),
        "Average Client BMIs": 'function() {{ return {tpl}.average()(["bmi"]) }}'.format(**dic)



    }
    opts.sorters = {
        'age': 'function(a,b){return b-a;}', # sort backwards
        'region': """{sortAs}(["southwest", "southeast",
                                "northwest", "northeast"])""".format(**dic)
    }
    return p

pivot_table()

PivotUIBox(children=(HBox(children=(Button(description='Save', layout=Layout(width='100px'), style=ButtonStyle…